In [ ]:
%matplotlib nbagg
import numpy as np
import json
import os, math
import pandas as pd
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from IPython.display import clear_output, display

from alvra_tools import *
import glob

In [ ]:
scan_name ="Na2SO4_monoscan_1"
#scan_name ="MB_monoscan_11"
DIR_json = "/sf/alvra/data/p18442/res/scan_info/"
DIR = "/sf/alvra/data/p18442/raw/scan_data/" + scan_name + "/"             #use this for a scan

#######################################################################

files_scan = glob.glob(DIR_json + scan_name + '.*')
print (files_scan)
scan_range = '.'+files_scan[-1].split('.')[-2]
print (scan_range)

!ls -lah -rt $DIR | grep BSREAD.h5
!hostname

# Load one step and check I0 filtering

In [ ]:
f ="run_002140.BSREAD.h5"
fn = DIR + f

(DataTrans_shot, DataFluo_shot, IzeroFEL_shot, Laser, Energy_shot, PulseIDs) = load_FEL_scans(fn, channel_energy)

IzeroFEL_shot_raw = IzeroFEL_shot.copy()
DataTrans_shot_raw = DataTrans_shot.copy()
DataFluo_shot_raw = DataFluo_shot.copy()

print ("Energy = {} eV".format(Energy_shot.mean()))
print ("Median value Izero (all) = {}".format(np.median(IzeroFEL_shot_raw)))
print ("Median value fluo (all) = {}".format(np.median(DataFluo_shot_raw)))

fig = plt.figure(constrained_layout=True, figsize=(9,5))
plt.suptitle(scan_name +"/"+ f.split('.')[0], y = 1)
gs = fig.add_gridspec(2, 2)

f_ax1 = fig.add_subplot(gs[0, :-1])
f_ax1.set_title('Fluo')
plt.hist(DataFluo_shot, bins =np.arange(-0.5,2,0.01), facecolor='red', label="fluo", alpha=0.5)
plt.grid()
plt.legend(loc="best")

f_ax2 = fig.add_subplot(gs[1, :-1])
f_ax2.set_title('I_zero')
plt.hist(IzeroFEL_shot, bins =np.arange(0,0.85,0.01), facecolor='red', label='Izero', alpha=0.5)
plt.grid()
plt.legend(loc="best")

f_ax3 = fig.add_subplot(gs[0:, 1])
f_ax3.set_title('correlation')
plt.plot(IzeroFEL_shot, DataFluo_shot, color = 'red',  linestyle='None', label='correlation',marker='.')
plt.grid()
plt.legend(loc="best")
plt.xlabel("I_zero (PBPS122)")
plt.ylabel ("Fluorescence (PIPS)")

plt.show()

# Load mono scan

### First, without any filtering of the data, check data distributions with quantiles 

In [ ]:
quantiles = [0.5, 0.1, 0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9]

######################################################33
json_file = DIR_json + scan_name + scan_range + ".json"
print (json_file)

with open(json_file) as file:
    data = json.load(file)
numFiles = len(data['scan_files'])
#numFiles = 35

print ("Processing",numFiles,"files")

DataFluo = []
Energy_eV = []

for i in range(0,numFiles):             
    fn = str(data['scan_files'][i][0])    
    exists = os.path.isfile(fn)
    if exists: #and i!=0:
        clear_output(wait=True)
        print("step",i+1,"of",numFiles,": Processing %s" %(str(data['scan_files'][i][0])))
        
        (DataTrans_shot, DataFluo_shot, IzeroFEL_shot, Laser, Energy_shot, PulseIDs) = load_FEL_scans(fn, channel_energy)
        
        print ("Loaded {} shots".format(len(DataFluo_shot)))
    
        DataFluo_shot_norm = DataFluo_shot / IzeroFEL_shot
        
        df_fluo = pd.DataFrame(DataFluo_shot_norm)
              
        DataFluo.append(np.quantile(df_fluo, quantiles))        
        Energy_shot = [x for x in Energy_shot if (np.abs(x) > 0)]
        Energy_eV.append(np.mean(Energy_shot))
    
DataFluo = np.asarray(DataFluo)
Energy_eV = np.asarray(Energy_eV)

print ("Job done!")

In [ ]:
plt.figure(figsize = (7,5))

plt.plot(Energy_eV, DataFluo[:,0],label='median',marker='.' )
plt.fill_between(Energy_eV,np.array(DataFluo[:,4]),np.array(DataFluo[:,5]), label='20%', alpha = 0.3)
plt.fill_between(Energy_eV,np.array(DataFluo[:,3]),np.array(DataFluo[:,6]), label='40%', alpha = 0.3)
plt.fill_between(Energy_eV,np.array(DataFluo[:,2]),np.array(DataFluo[:,7]), label='60%', alpha = 0.3)
plt.fill_between(Energy_eV,np.array(DataFluo[:,1]),np.array(DataFluo[:,8]), label='80%', alpha = 0.3)
plt.grid()

plt.legend(loc="best")
plt.ylabel("XAS Fluo")
plt.xlabel("Energy (eV)")
plt.title(json_file.split('scan_info')[1], fontsize=12)

plt.show()

### Now choose thresholds and quantile

In [ ]:
min_I0 = 0.01
min_fluo = 0.001
quantile_corr = 0.5

########################################################################
json_file = DIR_json + scan_name + scan_range + ".json"
print (json_file)

with open(json_file) as file:
    data = json.load(file)
numFiles = len(data['scan_files'])
#numFiles = 35

print ("Processing",numFiles,"files")

DataFluo = []
Energy_eV = []

for i in range(0,numFiles):             
    filename = str(data['scan_files'][i][0])    
    exists = os.path.isfile(filename)
    if exists: #and i!=0:
        clear_output(wait=True)
        print("step",i+1,"of",numFiles,": Processing %s" %(str(data['scan_files'][i][0])))
        
        (DataTrans_shot, DataFluo_shot, IzeroFEL_shot, Laser, Energy_shot, PulseIDs) = load_FEL_scans(filename, channel_energy)
        
        print ("Loaded {} shots".format(len(DataFluo_shot)))
        
        threshold_FEL = IzeroFEL_shot > min_I0
        threshold_fluo = DataFluo_shot > min_fluo
        thresholds_filter = threshold_FEL & threshold_fluo
        DataFluo_shot = DataFluo_shot[thresholds_filter]
        IzeroFEL_shot = IzeroFEL_shot[thresholds_filter]
        
        print ("survived {} shots".format(len(DataFluo_shot)))
        
        DataFluo_shot_norm = DataFluo_shot / IzeroFEL_shot
        
        med = np.median(DataFluo_shot_norm)
        qnt_low = np.quantile(DataFluo_shot_norm, 0.5 - quantile_corr/2)
        qnt_high = np.quantile(DataFluo_shot_norm, 0.5 + quantile_corr/2)
        condition_corr_low = DataFluo_shot_norm > qnt_low
        condition_corr_high = DataFluo_shot_norm < qnt_high
        correlation_filter = condition_corr_low & condition_corr_high 
        DataFluo_shot_norm = DataFluo_shot_norm[correlation_filter]
        
        print ("survived {} shots".format(len(DataFluo_shot_norm)))
        
        df_fluo = pd.DataFrame(DataFluo_shot_norm)
        DataFluo.append(np.quantile(df_fluo, [0.5, 0, 1]))
        
        Energy_shot = [x for x in Energy_shot if (np.abs(x) > 0)]
        Energy_eV.append(np.mean(Energy_shot))
    
DataFluo = np.asarray(DataFluo)                  
Energy_eV = np.asarray(Energy_eV)

print ("Job done!")

In [ ]:
plt.figure(figsize = (7,5))

XAS = DataFluo[:,0]
err_low = DataFluo[:,1]
err_high = DataFluo[:,2]

plt.plot(Energy_eV, XAS, label='{}%'.format(quantile_corr*100),marker='.')
plt.fill_between(Energy_eV, err_low, err_high, color='lightblue')
plt.xlabel("Energy (eV)")
plt.ylabel ("XAS (I0 norm)")
plt.gca().set_title('XAS (fluo)')
plt.legend(loc="best")
plt.title(json_file.split('scan_info')[1], fontsize=12)
plt.grid()

plt.show()